In [97]:
import pickle
import pandas as pd
import os
import matplotlib.pyplot as plt
import tldextract
import numpy as np
from torch_geometric.data import HeteroData
from sklearn.preprocessing import LabelEncoder
from torch_geometric.datasets import MovieLens
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.utils import to_networkx
import networkx as nx

In [122]:
col_names_mentions = ["GlobalEventID","EventTimeDate","MentionTimeDate","MentionType",
        "MentionSourceName","MentionIdentifier","SentenceID","Actor1CharOffset","Actor2CharOffset",
        "ActionCharOffset","InRawText","Confidence","MentionDocLen","MentionDocTone",
        "SRCLC", "ENG"] # mention columns

col_names_events = ["GlobalEventID","Day","MonthYear","Year","FractionDate",
                            "Actor1Code","Actor1Name","Actor1CountryCode","Actor1KnownGroupCode","Actor1EthnicCode",
                            "Actor1Religion1Code","Actor1Religion2Code","Actor1Type1Code","Actor1Type2Code","Actor1Type3Code",
                            "Actor2Code","Actor2Name","Actor2CountryCode","Actor2KnownGroupCode","Actor2EthnicCode",
                            "Actor2Religion1Code","Actor2Religion2Code","Actor2Type1Code","Actor2Type2Code","Actor2Type3Code",
                            "IsRootEvent","EventCode","EventBaseCode","EventRootCode","QuadClass",
                            "GoldsteinScale","NumMentions","NumSources","NumArticles","AvgTone",
                            "Actor1Geo_Type","Actor1Geo_Fullname","Actor1Geo_CountryCode","Actor1Geo_ADM1Code","Actor1Geo_ADM2Code",
                            "Actor1Geo_Lat","Actor1Geo_Long","Actor1Geo_FeatureID","Actor2Geo_Type","Actor2Geo_Fullname",
                            "Actor2Geo_CountryCode","Actor2Geo_ADM1Code","Actor2Geo_ADM2Code","Actor2Geo_Lat","Actor2Geo_Long",
                            "Actor2Geo_FeatureID","ActionGeo_Type","ActionGeo_Fullname","ActionGeo_CountryCode","ActionGeo_ADM1Code",
                            "ActionGeo_ADM2Code","ActionGeo_Lat","ActionGeo_Long","ActionGeo_FeatureID","DATEADDED",
                            "SOURCEURL"] # event columns

with open("urls_to_download_fr.pkl", 'rb') as file: # label of articles (0 if fake or not present in MBFS)
    df_mentions = pickle.load(file)
    
with open("labeled sources.pkl", 'rb') as file: # label of sources (0 if fake or not present in MBFS)
    df_sources = pickle.load(file)
    
with open("mixt_labeled_sources.pkl", 'rb') as file: # label of sources (nan if not present in MBFS)
    df_sources_mixte = pickle.load(file)
    
with open("mixt_labeled_articles.pkl", 'rb') as file: # label of articles (nan if not present in MBFS)
    df_articles_mixte = pickle.load(file)

MBFS = pd.read_csv("../mediabiasfactcheck/mediabiasfactcheck.csv")

df_test = pd.read_csv("gdelt_data/20231001000000.mentions.CSV", delimiter='\t', names=col_names_mentions)

df_test_event = pd.read_csv("gdelt_data_event/20231001000000.export.CSV", delimiter='\t', names=col_names_events)


In [8]:
pd.set_option('display.max_columns', None)

In [ ]:
#TODO

# Data Analysis
    # number of articles by source and compare this stat with the fact that they could be fake
    # count the number of labelized elements in each relevant label dataset
    # number of csv mentions/events in october 2023
    # number of csv events in october 2023
    # number of articles/sources in total
    # number of nans in each columns
    # look at sources type in mention and evaluate impact
    
    
# Data Processing
    # remove irrelevant columns/nans?
    # put them in the correct format for graph


## Data Analysis

In [5]:
df_sources_mixte["is_fake"].value_counts(dropna = False) /len(df_sources_mixte["is_fake"])

is_fake
NaN    0.920925
0.0    0.076651
1.0    0.002423
Name: count, dtype: float64

In [7]:
df_articles_mixte["is_fake"].value_counts(dropna = False)/len(df_articles_mixte["is_fake"])

is_fake
NaN    0.744008
0.0    0.248798
1.0    0.007194
Name: count, dtype: float64

In [27]:
MBFS.isna().mean()

url                    0.043817
name                   0.000269
image_pseudoscience    0.900000
image_factual          0.081720
image_conspiracy       0.908333
image_bias             0.149462
freedom_rank           0.593817
country                0.546237
dtype: float64

In [56]:
df_sources_mixte[df_sources_mixte["is_fake"] == 1.0].head()

,links,is_fake
15,infiniteunknown.net,1.0
54,sott.net,1.0
102,dailymail.co.uk,1.0
131,wordpress.com,1.0
163,oann.com,1.0


In [104]:
df_articles_mixte[df_articles_mixte["links"].str.contains("oann.com")]

,links,is_fake
341,https://www.oann.com/newsroom/death-toll-rises...,1.0
688,https://www.oann.com/newsroom/man-accused-of-k...,1.0
29234,https://www.oann.com/newsroom/dem-rep-jamaal-b...,1.0
29364,https://www.oann.com/newsroom/girl-9-disappear...,1.0
36554,https://www.oann.com/newsroom/breonna-taylors-...,1.0
...,...,...
1908385,https://www.oann.com/newsroom/israeli-military...,1.0
1908605,https://www.oann.com/business/us-seeks-to-bloc...,1.0
1925791,https://www.oann.com/newsroom/person-of-intere...,1.0
1926135,https://www.oann.com/newsroom/fbi-chief-christ...,1.0


In [97]:
MBFS = MBFS.dropna(subset=['url'])
MBFS[MBFS["url"].str.contains("oann.com")]


,url,name,image_pseudoscience,image_factual,image_conspiracy,image_bias,freedom_rank,country
468,https://www.oann.com,One America News Network (OAN),NaN,MBFCLow.png,NaN,right.?.?.?.png,NaN,NaN


In [3]:
csv_count = 0
total = 0
nan = 0

for root, _, filenames in os.walk("gdelt_data"):
    for filename in filenames:
        if filename.endswith(".CSV"):
            csv_count += 1
            temp = pd.read_csv(os.path.join(root, filename), delimiter='\t', names=col_names_mentions)
            nan += temp.isna().mean()
            total+=len(temp)

print("csv_count for mentions: "+str(csv_count))
print("total number of lines for mentions: "+str(total))
print("percentage of nan for each column: "+str(nan/csv_count))

csv_count for mentions: 2974
total number of lines for mentions: 14485980
percentage of nan for each column: GlobalEventID        0.000000e+00
EventTimeDate        0.000000e+00
MentionTimeDate      0.000000e+00
MentionType          0.000000e+00
MentionSourceName    8.460116e-07
MentionIdentifier    0.000000e+00
SentenceID           0.000000e+00
Actor1CharOffset     0.000000e+00
Actor2CharOffset     0.000000e+00
ActionCharOffset     0.000000e+00
InRawText            0.000000e+00
Confidence           0.000000e+00
MentionDocLen        0.000000e+00
MentionDocTone       0.000000e+00
SRCLC                1.000000e+00
ENG                  1.000000e+00
dtype: float64


In [43]:
csv_count = 0
total = 0
nan = 0

for root, _, filenames in os.walk("gdelt_data_event"):
    for filename in filenames:
        if filename.endswith(".CSV"):
            csv_count += 1
            temp = pd.read_csv(os.path.join(root, filename), delimiter='\t', names=col_names_events) 
            total+=len(temp)
            nan += temp.isna().mean()


print("csv_count for events: "+str(csv_count))
print("total number of lines for events: "+str(total))
print("percentage of nan for each column: "+str(nan/csv_count))

csv_count for events: 2974
total number of lines for events: 4606957
percentage of nan for each column: GlobalEventID          0.000000
Day                    0.000000
MonthYear              0.000000
Year                   0.000000
FractionDate           0.000000
                         ...   
ActionGeo_Lat          0.024224
ActionGeo_Long         0.024117
ActionGeo_FeatureID    0.023964
DATEADDED              0.000000
SOURCEURL              0.000000
Length: 61, dtype: float64


In [4]:
value_count = 0
csv_count = 0

for root, _, filenames in os.walk("gdelt_data"):
    for filename in filenames:
        if filename.endswith(".CSV"):
            csv_count += 1
            temp = pd.read_csv(os.path.join(root, filename), delimiter='\t', names=col_names_mentions)
            value_count += temp["MentionType"].value_counts()
            if len(temp["MentionType"].value_counts())>1:
                print("omg")

            
print(value_count)


MentionType
1    14485980
Name: count, dtype: int64


## PreProcessing -> put in graph format

#### MovieLens

In [4]:
dataset = MovieLens(root='data/MovieLens', transform=NormalizeFeatures())

c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# print(f'Number of training nodes: {data.train_mask.sum()}')
# print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: MovieLens():
Number of graphs: 1
Number of features: {'movie': 404, 'user': 0}

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836],
  }
)
Number of nodes: 10352
Number of edges: 100836
Average node degree: 9.74
Has isolated nodes: True
Has self-loops: False
Is undirected: False


In [21]:
# data["user", "rates", "movie"].edge_index
# data["movie"]
data["movie"]

{'x': tensor([[0.0016, 0.0028, 0.0028,  ..., 0.0024, 0.0024, 0.0024],
        [0.0014, 0.0038, 0.0022,  ..., 0.0024, 0.0024, 0.0024],
        [0.0015, 0.0026, 0.0019,  ..., 0.0024, 0.0024, 0.0024],
        ...,
        [0.0014, 0.0027, 0.0023,  ..., 0.0024, 0.0024, 0.0024],
        [0.0028, 0.0026, 0.0027,  ..., 0.0024, 0.0024, 0.0024],
        [0.0020, 0.0023, 0.0024,  ..., 0.0024, 0.0024, 0.0024]])}

In [ ]:
# g_nx = to_networkx(data)

# # Visualize the graph using NetworkX
# pos = nx.spring_layout(g_nx)
# nx.draw(g_nx, pos, with_labels=True, node_color='skyblue', font_weight='bold', font_color='black', node_size=800, edge_color='gray')

# # Show the plot
# plt.show()

#### Creating Graph

In [51]:
# label_encoder = LabelEncoder()
# df_test['MentionSourceName'] = label_encoder.fit_transform(df_test['MentionSourceName'])
# label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# label_encoder_article = LabelEncoder()
# df_test['MentionIdentifier'] = label_encoder_article.fit_transform(df_test['MentionIdentifier'])
# label_mapping_article = dict(zip(label_encoder_article.classes_, label_encoder_article.transform(label_encoder_article.classes_)))

# label_encoder_event = LabelEncoder()
# df_test['GlobalEventID'] = label_encoder_event.fit_transform(df_test['GlobalEventID'])
# label_mapping_event = dict(zip(label_encoder_event.classes_, label_encoder_event.transform(label_encoder_event.classes_)))
# df_test_event['GlobalEventID'] = df_test_event['GlobalEventID'].map(label_mapping_event)

In [123]:
# Encoding and creating the map for df_sources

label_encoder_source = LabelEncoder()
df_sources_mixte['links'] = label_encoder_source.fit_transform(df_sources_mixte['links'])
label_mapping_source = dict(zip(label_encoder_source.classes_, label_encoder_source.transform(label_encoder_source.classes_)))
# df_sources_mixte['links'] = df_sources_mixte['links'].map(label_mapping_source)

df_sources_mixte_sorted = df_sources_mixte.sort_values(by="links").set_index("links")
df_sources_mixte_sorted = df_sources_mixte_sorted.reset_index(drop=False)
mapping_source = df_sources_mixte_sorted["links"]
df_sources_mixte_sorted["random"] = np.random.rand(14442)
y = df_sources_mixte_sorted["is_fake"]
df_sources_mixte_sorted = df_sources_mixte_sorted[["random"]]

In [125]:
# Encoding and creating the map for df_article

df_article = pd.DataFrame(df_test["MentionIdentifier"])

label_encoder_article = LabelEncoder()
df_article['MentionIdentifier'] = label_encoder_article.fit_transform(df_article['MentionIdentifier'])
label_mapping_article = dict(zip(label_encoder_article.classes_, label_encoder_article.transform(label_encoder_article.classes_)))

df_article_sorted = df_article.sort_values(by="MentionIdentifier").set_index("MentionIdentifier")
df_article_sorted = df_article_sorted.reset_index(drop=False)
mapping_article = df_article_sorted["MentionIdentifier"]
df_article_sorted["random"] = np.random.rand(1855)
df_article_sorted = df_article_sorted[["random"]]

In [127]:
# Encoding and creating the map for df_event

label_encoder_event = LabelEncoder()
df_test_event['GlobalEventID'] = label_encoder_event.fit_transform(df_test_event['GlobalEventID'])
label_mapping_event = dict(zip(label_encoder_event.classes_, label_encoder_event.transform(label_encoder_event.classes_)))
# df_test_event['GlobalEventID'] = df_test_event['GlobalEventID'].map(label_mapping_event)
df_test_event_sorted = df_test_event.sort_values(by="GlobalEventID").set_index("GlobalEventID")
df_test_event_sorted = df_test_event_sorted.reset_index(drop=False)
mapping_event = df_test_event_sorted["GlobalEventID"]
df_test_event_sorted = df_test_event_sorted.drop("GlobalEventID",axis = 1)
df_test_event_sorted = df_test_event_sorted.drop("GlobalEventID",axis = 1)

In [134]:
# Encoding all the keys in the mention dataset

df_test['MentionIdentifier'] = df_test['MentionIdentifier'].map(label_mapping_article)
df_test['MentionSourceName'] = df_test['MentionSourceName'].map(label_mapping_source)
df_test['GlobalEventID'] = df_test['GlobalEventID'].map(label_mapping_event)


In [135]:
est_source_de = df_test[["MentionSourceName","MentionIdentifier"]]

In [142]:
mapping_source

0            0
1            1
2            2
3            3
4            4
         ...  
14437    14437
14438    14438
14439    14439
14440    14440
14441    14441
Name: links, Length: 14442, dtype: int32

In [143]:
# Mapping articles and sources to create edges of type "df_test_event_sorted"

article_map = mapping_article.reset_index().set_index("MentionIdentifier").to_dict()
est_source_de["MentionIdentifier"] = est_source_de["MentionIdentifier"].map(article_map["index"]).astype(int)

source_map = mapping_source.reset_index().set_index("links").to_dict()
est_source_de["MentionSourceName"] = est_source_de["MentionSourceName"].map(source_map["index"]).astype(int)

C:\Users\matth\AppData\Local\Temp\ipykernel_46416\505668169.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  est_source_de["MentionSourceName"] = est_source_de["MentionSourceName"].map(source_map["index"]).astype(int)


In [146]:
edge_est_source_de = est_source_de[["MentionSourceName", "MentionIdentifier"]].values.transpose()
edge_est_source_de # [2 x num_edges] 

array([[  461,  5199,  7027, ..., 10925, 10925, 10925],
       [  464,   879,   337, ...,  1395,  1395,  1395]])

In [ ]:
# Using only the first csv of events mentions and sources to start


data = HeteroData()
data['articles'] = df_article_sorted.to_numpy()
data['sources'] = df_sources_mixte_sorted.to_numpy()
data['events'].x = df_test_event_sorted.to_numpy()

data['event', 'mentionné', 'article'].edge_attr

data['event', 'mentionné', 'article'].edge_index
data['source', 'est_source_de', 'article'].edge_index = edge_est_source_de